In [1]:
# Tries all permutations of [957496696762772407663, 'ETH', 'BTC', 'XRP', 'Phemex']
# with different:
#
# 1) character transformation functions (e.g. uppercase, lowercase)
# 2) character encoding function (e.g. ascii code, alphabetical index)
# 3) word to number mapping functions (e.g. sum all chars, concatenate chars)
# 4) reduce functions to combine numbers into a Bitcoin private key
#    (e.g. multiply, concatenate, hex then concatenate, etc.)

from ipynb.fs.full.utils import test_hex, to_hex
from itertools import permutations, combinations, product

from functools import reduce

In [2]:
strings = ['ETH', 'BTC', 'XRP', 'Phemex']

In [3]:
prime21 = 957496696762772407663

In [4]:
# naive solution (use 21 digit prime as private key)
test_hex(to_hex(prime21))

False

In [5]:
32 - len(to_hex(prime21))/2 # missing bytes (bitcoin priv key is 32 bytes)

23.0

In [6]:
segments = [prime21] + strings
segments

[957496696762772407663, 'ETH', 'BTC', 'XRP', 'Phemex']

In [7]:
permuted_segments = list(permutations(segments))

In [8]:
def identity(segment):
    return segment

# ignores phemex segment
def ignore_phemex(segment):
    if segment == 'Phemex':
        return None
    return segment

In [9]:
# decorator
def segment_is_string(func):
    def wrapper(segment):
        if (not isinstance(segment, str)):
            return segment
        return func(segment)
    return wrapper

In [10]:
# decorator
def segment_is_list(func):
    def wrapper(segment):
        if (not isinstance(segment, list)):
            return segment
        return func(segment)
    return wrapper

In [11]:
@segment_is_string
def upper(segment):
    return segment.upper()

@segment_is_string
def lower(segment):
    return segment.lower()
    

In [12]:
def char_encoding_alpha(c):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    return alphabet.find(c.lower()) + 1

In [13]:
def char_encoding_ascii(c):
    return ord(c)

In [14]:
@segment_is_list
def word_encoding_sum(segment):
    return sum(segment)

In [15]:
word_encoding_sum([char_encoding_alpha(c) for c in 'aB'])

3

In [16]:
word_encoding_sum([char_encoding_ascii(c) for c in 'aB'])

163

In [17]:
@segment_is_list
def word_encoding_concat(segment):
    return int(''.join([str(n) for n in segment]))

In [18]:
word_encoding_concat([char_encoding_ascii(c) for c in 'aB'])

9766

In [19]:
word_encoding_concat([char_encoding_alpha(c) for c in 'aB'])

12

In [20]:
def encode_chars(fn):
    @segment_is_string
    def wrapper(segment):
        return [fn(c) for c in segment]
    return wrapper

In [21]:
def prod(lst):
    return product(*lst)
list(prod([['id', 'ignore_phemex'], ['middle'], ['id', 'upper']]))

[('id', 'middle', 'id'),
 ('id', 'middle', 'upper'),
 ('ignore_phemex', 'middle', 'id'),
 ('ignore_phemex', 'middle', 'upper')]

In [22]:
def compose(*functions):
    return reduce(lambda x, y: (lambda arg: x(y(arg))), functions)
compose(lambda x: x*2, lambda x: x+5)(10)

30

In [23]:
def apply_pipeline(p):
    def wrapper(segments):
        return [p(s) for s in segments]
    return wrapper

def create_pipelines(lst):
    pipelines = list(reversed(lst))
    pipelines = list(prod(pipelines))
    pipelines = [compose(*p) for p in pipelines]
    pipelines = [apply_pipeline(p) for p in pipelines]
    
    return pipelines

In [24]:
pipelines = create_pipelines([
    [identity, ignore_phemex],
    [identity, upper, lower],
    [encode_chars(char_encoding_alpha), encode_chars(char_encoding_ascii)],
    [word_encoding_concat, word_encoding_sum],
])

In [25]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [26]:
# apply all pipelines to all segments
res = [[p(segments) for p in pipelines] for segments in permuted_segments]
res = flatten(res)
res = [list(filter(None, x)) for x in res]
res

[[957496696762772407663, 5208, 2203, 241816, 168513524],
 [957496696762772407663, 5208, 2203, 241816],
 [957496696762772407663, 5208, 2203, 241816, 168513524],
 [957496696762772407663, 5208, 2203, 241816],
 [957496696762772407663, 5208, 2203, 241816, 168513524],
 [957496696762772407663, 5208, 2203, 241816],
 [957496696762772407663, 698472, 668467, 888280, 80104101109101120],
 [957496696762772407663, 698472, 668467, 888280],
 [957496696762772407663, 698472, 668467, 888280, 807269776988],
 [957496696762772407663, 698472, 668467, 888280],
 [957496696762772407663, 101116104, 9811699, 120114112, 112104101109101120],
 [957496696762772407663, 101116104, 9811699, 120114112],
 [957496696762772407663, 33, 25, 58, 71],
 [957496696762772407663, 33, 25, 58],
 [957496696762772407663, 33, 25, 58, 71],
 [957496696762772407663, 33, 25, 58],
 [957496696762772407663, 33, 25, 58, 71],
 [957496696762772407663, 33, 25, 58],
 [957496696762772407663, 225, 217, 250, 615],
 [957496696762772407663, 225, 217, 250

In [27]:
def mult(nums):
    return reduce(lambda a,b : a*b, nums,1)
def concat(nums):
    return int(reduce(lambda a,b : a+str(b), nums, ''))
concat([22, 44, 12])

224412

In [28]:
# first convert segment to hex, then concat
def hex_concat(nums):
    return int(reduce(lambda a,b : a+to_hex(b), nums, ''), 16)
to_hex(hex_concat([int(s, 16) for s in ['de', 'ad', 'be', 'ef']]))

'deadbeef'

In [29]:
combine_fns = [mult, sum, concat, hex_concat]

In [30]:
for segments in res:
    for combine in combine_fns:
        test_hex(to_hex(combine(segments)))